In [47]:
import os
import numpy as np
import pandas as pd
import math as m

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.stattools import adfuller
import seaborn as sns

from scipy import stats

import matplotlib.pyplot as plt
import matplotlib as mpl

from scipy import stats
from scipy.fftpack import fft, rfft, fftfreq
from scipy import signal



from stargazer.stargazer import Stargazer
from IPython.core.display import HTML




In [48]:
os.chdir('/Users/rgreen/Documents/Github/NDVI_Projection/')

oromia = pd.ExcelFile('oromia.xlsx')
arsi = pd.read_excel(oromia, 'arsi')
bale = pd.read_excel(oromia, 'bale')
borena = pd.read_excel(oromia, 'borena')
guji = pd.read_excel(oromia, 'guji')
westarsi = pd.read_excel(oromia, 'westarsi')


arsi.insert(0, 'Time', np.linspace(1,585,585))
bale.insert(0, 'Time', np.linspace(1,585,585))
borena.insert(0, 'Time', np.linspace(1,585,585))
guji.insert(0, 'Time', np.linspace(1,585,585))
westarsi.insert(0, 'Time', np.linspace(1,585,585))

In [49]:
#dekadal data (P, LST, ET)    
ddf = pd.DataFrame()
ddf['D_NDVI'] = arsi.NDVI.diff()[1:]
ddf = ddf.reset_index(drop=True)
#L_NDVI = arsi.NDVI.shift(-1) #lag
ddf['NDVI_P'] = ((max(arsi.NDVI) - (arsi.NDVI[:-1]))* (arsi.P[:-1]))
ddf['NDVI_LST'] = (((arsi.NDVI[:-1]) - min(arsi.NDVI))* (arsi.LST[:-1])).shift(4) #need to shift back lags, use shift not index
ddf['NDVI_ET'] = (((arsi.NDVI[:-1]) - min(arsi.NDVI))* (arsi.ET[:-1])).shift(4)



In [50]:
#shift is (n-1) dekads, ex. shift(4) is 5 dekads lag

#mask
mask_lst = ~np.isnan(ddf.N_LST) & ~np.isnan(ddf.D_NDVI)
mask_et = ~np.isnan(ddf.N_ET) & ~np.isnan(ddf.D_NDVI)

#stats.lingress (X,Y)
slope1, intercept1, r1, p1, std1 = stats.linregress(ddf.N_P, ddf.D_NDVI)
line1 = slope1*ddf.N_P+intercept1
print("r-squared: %f" % r1**2)
slope2, intercept2, r2, p2, std2 = stats.linregress(ddf.N_LST[mask_lst], ddf.D_NDVI[mask_lst])
line2 = slope2*ddf.N_LST+intercept2
print("r-squared: %f" % r2**2)
slope3, intercept3, r3, p3, std3 = stats.linregress(ddf.N_ET[mask_et], ddf.D_NDVI[mask_et])
line3 = slope3*ddf.N_ET+intercept3
print("r-squared: %f" % r3**2)

fig, (ax1, ax2, ax3) = plt.subplots(3, sharey =True)
ax1.scatter(ddf.N_P, ddf.D_NDVI, color = 'darkcyan')
ax1.plot(ddf.N_P, line1, color = 'k')
ax1.set(xlabel = r'$(NDVI_{max} - NDVI_{t-1})*P_{t-1}$', ylabel = '')
ax1.text(16, -0.04, r'y = 0.005x + 0.02', fontsize=8)
ax1.text(16, -0.05, r'$r^2$ = 0.528', fontsize=8)
ax2.scatter(ddf.N_LST, ddf.D_NDVI, color = 'forestgreen')
ax2.set(xlabel = r'$(NDVI_{t-5} - NDVI_{min})*LST_{t-5}$', ylabel = '')
ax2.plot(ddf.N_LST, line2, color = 'k')
ax2.text(9, 0.07, r'y = -0.005x + 0.038', fontsize=8)
ax2.text(9, 0.06, r'$r^2$ = 0.386', fontsize=8)
ax3.scatter(ddf.N_ET, ddf.D_NDVI, color = 'cornflowerblue')
ax3.set(xlabel = r'$(NDVI_{max} - NDVI_{t-5})*ET_{t-5}$', ylabel = '')
ax3.plot(ddf.N_ET, line3, color = 'k')
ax3.text(15, 0.07, r'y = -0.003x + 0.031', fontsize=8)
ax3.text(15, 0.06, r'$r^2$ = 0.414', fontsize=8)
fig.text(0.06, 0.5, r'$\Delta NDVI$', ha='center', va='center', rotation='vertical') #common ylabel
# =============================================================================
#     
#     sns.set(style="ticks", color_codes=True)
#     
#     fig = plt.figure()
#     sns.regplot(x=ddf.N_P, y=ddf.D_NDVI)
#     
#     g = sns.PairGrid(ddf, y_vars=["D_NDVI"], x_vars=["N_P", "N_LST", "N_ET"], height=4)
#     g.map(sns.regplot, color=".3")
#     
#     replacements = {'D_NDVI': r'$\Delta NDVI$', 'N_P': '(maxNDVI - NDVIt-1)*Pt-1',
#                 'N_LST': '(NDVIt-5 - minNDVI)*LSTt-5', 'N_ET': '(NDVIt-5 - minNDVI)*N_ETt-5'}
# 
#     for i in range(4):
#         for j in range(4):
#             xlabel = g.axes[i][j].get_xlabel()
#             ylabel = g.axes[i][j].get_ylabel()
#             if xlabel in replacements.keys():
#                 g.axes[i][j].set_xlabel(replacements[xlabel])
#             if ylabel in replacements.keys():
#                 g.axes[i][j].set_ylabel(replacements[ylabel])
#         
# =============================================================================

# =============================================================================
#    
#     r, p = stats.pearsonr(ddf.D_NDVI, ddf.N_P)
#     r, p = stats.pearsonr(ddf.D_NDVI, ddf.N_LST)
#     r, p = stats.pearsonr(ddf.D_NDVI, ddf.N_LST)
#     

AttributeError: 'DataFrame' object has no attribute 'N_LST'

In [51]:
#sm.OLS (Y,X)
#ordinary least squares regression fit
# =============================================================================
#     
# from IPython.core.display import HTML
# def short_summary(est):
#     return HTML(est.summary().tables[1].as_html())
# =============================================================================

model1 = ddf[['NDVI_P', 'NDVI_LST']]  #for multivariate regression ddf[['N_P', 'N_LST']]
model2 = ddf[['NDVI_P', 'NDVI_ET']]
model3 = ddf[['NDVI_LST', 'NDVI_ET']]
model4 = ddf[['NDVI_LST', 'NDVI_ET', 'NDVI_P']]
Y = ddf[['D_NDVI']]
model1 = sm.add_constant(model1)
model2 = sm.add_constant(model2)
model3 = sm.add_constant(model3)
model4 = sm.add_constant(model4)
est1 = sm.OLS(Y,model1, missing = 'drop').fit()
est2 = sm.OLS(Y,model2, missing = 'drop').fit()
est3 = sm.OLS(Y,model3, missing = 'drop').fit()
est4 = sm.OLS(Y,model4, missing = 'drop').fit()


stargazer = Stargazer([est1, est2, est3, est4])

/Users/rgreen/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [52]:


stargazer.dependent_variable_name('NDVI 1 Dekad Ahead Projection')
HTML(stargazer.render_html())
